In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
normal = pd.read_csv("C:/Users/tlseh/Desktop/WADI_14days_new.csv")

In [3]:
normal_drop = normal.drop(['Row','Date','Time'],axis='columns')

In [4]:
normal_drop_1 = normal_drop[0:1000] # 앞에 1000개만 우선 해보기 위해서 부분 선택
normal_drop_1 = normal_drop_1.dropna(axis=1) # 값이 없는 열 제거 여기서는 행 4개가 제거되었다.

In [5]:
normal_drop_1

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,...,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
1,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
2,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
3,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
4,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
996,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
997,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
998,174.213,0.541483,11.7854,482.401,0.286422,2.003600,0,0,40.8926,2,...,1,1,1,1,1,1,1,63.9204,1,0.68


In [6]:
normal_drop_1.isnull().sum().sum() # 성분에서 NaN이 있으면 1이상의 값이 나오게 된다. pd.DataFrame.isnull().sum().sum()

0

In [7]:
from sklearn import preprocessing       # 정규화를 반드시 진행해야 하는 건지?
min_max_scaler = preprocessing.MinMaxScaler()

x = normal_drop_1.values
x_scaled = min_max_scaler.fit_transform(x)
normal_drop_1_scaled = pd.DataFrame(x_scaled)

print(normal_drop_1_scaled)

          0         1         2         3         4         5    6    7    \
0    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
1    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
2    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
3    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
4    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
..        ...       ...       ...       ...       ...       ...  ...  ...   
995  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
996  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
997  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
998  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   
999  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   

          8    9    ...  113  114  115  116  117  118  119       120  121  

In [8]:
normal_drop_1 = np.array(normal_drop_1).astype('float') # Dataframe을 numpy로 변환

In [9]:
normal_drop_1.shape
print("before transpose :{}".format(normal_drop_1.shape)) # 데이터셋 입력전 rnn 모양을 맞춰주기 위해서 행열 transpose
normal_drop_1 = np.transpose(normal_drop_1)
print("after transpose :{}".format(normal_drop_1.shape))

before transpose :(1000, 123)
after transpose :(123, 1000)


In [10]:
normal_drop_1 = torch.from_numpy(normal_drop_1).unsqueeze(0) # 배치 차원을 추가 해주기 위해서 첫번째 위치에 차원 추가

In [11]:
normal_drop_1.shape

torch.Size([1, 123, 1000])

In [12]:
def Window(data,window,step):   # window 구현을 어떻게 해야 할지 몰라서 함수로 구현
    batch_size = (data.shape[-1] - window) // step
    Window_complete = torch.tensor([])

    for index in range(batch_size):
        x = torch.clone(data[:,:, step * index : window + step * index])
        Window_complete = torch.cat((Window_complete,x),dim=0)

    return Window_complete 

In [13]:
data = Window(normal_drop_1,window=5,step=3)

In [14]:
data.shape

torch.Size([331, 123, 5])

In [15]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [16]:
class CustomDataset(Dataset):   # 데이터셋에 굳이 넣어야 하는건지?
  def __init__(self,data):
      self.data = data

  def __getitem__(self, idx): 
    return self.data[idx]

  def __len__(self): 
    return len(self.data)


In [17]:
dataset = CustomDataset(data)

In [18]:
INPUT_SIZE = dataset.data.shape[-1]
SEQ_LEN = dataset.data.shape[-2]
BATCH_SIZE = 3

In [19]:
type(INPUT_SIZE)

int

In [81]:
dataloader = DataLoader(dataset,batch_size=BATCH_SIZE, shuffle=False,drop_last=True)

In [21]:
from torch import nn, optim

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim=32):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = embedding_dim * 2
        self.embedding_dim = embedding_dim 

        
        self.rnn1 = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_dim,num_layers=1,batch_first=True)
        self.rnn2 = nn.LSTM(input_size=self.hidden_dim,hidden_size=self.embedding_dim,num_layers=1,batch_first=True)

    def forward(self, x):           # (batch size, 123, window)
        x = x.float()
        x,(_,_) = self.rnn1(x)
        x, (hidden,cell) = self.rnn2(x) # (batch size, 123, embedding_dim)

        return hidden

In [23]:
class Decoder(nn.Module):
    def __init__(self,input_size=32, output_size=INPUT_SIZE):
        super(Decoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = input_size * 2
        self.output_size = output_size

        self.rnn1 = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_dim,batch_first=True)
        self.rnn2 = nn.LSTM(input_size=self.hidden_dim,hidden_size=self.output_size,batch_first=True)

    def forward(self,x):
        x = x.permute(1,0,2)
        x = x.repeat(1,SEQ_LEN,1)
        x,(_,_) = self.rnn1(x)
        x,(_,_) = self.rnn2(x)
        return x

In [37]:
class Autoencoder(nn.Module):

  def __init__(self,input_size,output_size, embedding_dim=64):
    super(Autoencoder, self).__init__()

    self.encoder = Encoder(input_size, embedding_dim=32)
    self.decoder = Decoder(input_size=32, output_size=INPUT_SIZE)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [41]:
model = Autoencoder(5,1)

In [42]:
optimizer = torch.optim.Adam(model.parameters())

In [75]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.L1Loss(reduction='sum')

In [82]:
for i, batch in enumerate(dataloader):
        
    optimizer.zero_grad()
    
    output = model(batch)
    
    loss = criterion(output, batch)
    
    loss.backward()

    if i % 10 == 0:
        print("NOW index : {}, LOSS : {}".format(i, loss))

    optimizer.step()
    

NOW index : 0, LOSS : 82319.6015625
NOW index : 10, LOSS : 85268.0
NOW index : 20, LOSS : 86746.140625
NOW index : 30, LOSS : 87032.453125
NOW index : 40, LOSS : 86911.234375
NOW index : 50, LOSS : 86618.75
NOW index : 60, LOSS : 86228.875
NOW index : 70, LOSS : 62145.953125
NOW index : 80, LOSS : 84153.3828125
NOW index : 90, LOSS : 85851.640625
NOW index : 100, LOSS : 85959.5625


In [78]:
for i, batch in enumerate(dataloader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


In [29]:
xxx = 0
x = 0
model = Encoder(5)
for i,dom in enumerate(dataloader):
    x = model(dom)

    xxx += 1
    if xxx ==1:
        break

In [30]:
dom.shape

torch.Size([3, 123, 5])

In [31]:
x.permute(1,0,2).shape

torch.Size([3, 1, 32])

In [32]:
modu = Decoder()

In [35]:
x_final = modu(x)

In [36]:
x_final.shape

torch.Size([3, 123, 5])

In [38]:
aaa = Autoencoder(5,1)

In [39]:
fffff = aaa(dom)

In [40]:
fffff.shape

torch.Size([3, 123, 5])